# Module Import
## Module


* numpy : ndarray
* cv2 : image resize, 불러오기 등의 처리
* glob : 디렉토리 파일 탐색
* tensorflow : 모듈 설계

In [1]:
import numpy as np
import cv2
import sys
import os
import glob
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# Colab 드라이브 마운트
##### 이후 $ cd gdrive/MyDrive/경로.. 로 기본 디렉토리 수정하여 사용
##### 이미지 데이터를 가지고 와서 train data 라벨링 진행해야 함

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
cd gdrive/MyDrive/KT

/content/gdrive/MyDrive/KT


# 이미지 불러오기
##### glob.glob(path) 함수 결과를 sort하여 img데이터 들어오는 순서 고정
##### cv2.imread() 함수로 img를 읽어오게 됨
  - cv2.IMREAD_COLOR : 컬러 이미지 불러올 때 쓰는 옵션
##### cv2.resize() : 크기 조절
  - dsize를 고정함으로써 원하는 크기로 불러온 이미지 데이터 수정 가능

##### 256.0으로 나누어 float데이터 타입을 유지하면서 normalize





In [85]:
# get img list from path
files = sorted(glob.glob("image/*"))

In [86]:
files

['image/01.as', 'image/02.als', 'image/03.alskd', 'image/04.woe']

In [6]:
# read image from files list & reshape
# train_img shape : ( , 256, 256, 3)
train_img = np.array(np.array([cv2.resize(cv2.imread(file, cv2.IMREAD_COLOR), dsize=(256, 256),
                                 interpolation=cv2.INTER_LINEAR).astype(np.float64) for file in files]))

In [7]:
print(train_img.shape)
print(train_img[0].shape)
#print(train_img)

(4, 256, 256, 3)
(256, 256, 3)


In [11]:
train_label.shape

(4,)

In [10]:
train_label = np.array([0, 0, 1, 3])

In [8]:
# normalize
train_img = train_img / (256.0)

# Model Architecture

**코드가 조금 더러움 주의**

기본적으로 ResNet의 구조를 가지고 있는 NN으로 구성
## Hyperparameters
##### **수정 불가능한 hyperparameter**
  - input_img_shape : 256*256으로 고정
  - input_channel : RGB 3채널 이미지

##### **수정 가능한 hyperparmeter**
  - layer1_output_channel : layer1 의 output channel개수
  - layer1_kernel_size : ResNet 처음 conv kernel_size는 7*7 이용
  - layer2_output_channel : layer2 의 output channel개수 (== layer1_output_channel)
  - layer3_output_channel : layer3 의 output channel개수 (layer2_output_channel * 2)
  - res_kernel_size : resNet에서는 기본적으로 첫번째 conv 제외 3*3 kernel로 고정

## Layers
1. layer1 : 7*7 convolution + pooling
2. layer2 : res_unit1 * 3
3. layer3 : res_unit2 + res_unit3 * 3
4. flatten : 분류기에 넣기 위해 flatten진행
5. classifier : 분류기, FC layer이용, ResNet에서는 FC layer의 Weight개수가 많으므로 한개의 FC layer만 사용

### 1*1 Convolution
layer2에서 사용하는 res_unit2의 경우, 이미지 데이터가 들어갈 때 layer2_output_channel개로 들어가지만, 나올 때 layer3_output_channel로 나오게 되는데, channel개수를 맞추어주기 위해서 convolution진행.
이미지 크기도 res_unit2의 첫번째 convolution에서 stride (2, 2)로 진행하므로 반으로 줄어들게 되므로 1*1 convolution도 stride (2, 2)로 진행

In [9]:
# Hyperparameters
global input_img_shape, input_channel, layer1_output_channel, layer1_kernel_size, res_kernel_size, layer2_output_channel, layer3_output_channel
input_img_shape = (256, 256, 3)
input_channel = 3
layer1_output_channel = 12
layer1_kernel_size = (7, 7)
layer2_output_channel = 12
layer3_output_channel = 24

# overall residual units num = num_residual_units * num_residual_layers 
res_kernel_size = (3, 3)

In [78]:
class cnn_model(tf.keras.Model):
  global input_img_shape, input_channel, layer1_output_channel, layer1_kernel_size, res_kernel_size, layer2_output_channel, layer3_output_channel
  def __init__(self):
    super(cnn_model, self).__init__()
    self.num_residual_units = 3 # number of units per layer
    self.num_residual_layers = 2 # number of layer per model
    self.res_kernel_size = (3, 3)

    self.layer1 = models.Sequential(
        [layers.Conv2D(layer1_output_channel, layer1_kernel_size, strides=(2, 2), 
                       padding='same', input_shape=input_img_shape, activation='relu'),
         layers.MaxPool2D(pool_size=(2, 2))]
    )

    self.res_unit1 = models.Sequential(
        [layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.res_unit2 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, strides=(2, 2), padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.res_unit3 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )


    self.flatten_layer = layers.Flatten()

    self.classifier = models.Sequential(
        [layers.Dense(4)]
    )

    self.identity = layers.Conv2D(layer3_output_channel, (1, 1), padding='same', strides=(2, 2))

  def call(self, x):
    x = self.layer1(x)
    x_t = self.res_unit1(x)
    x = x + x_t
    x_t = self.res_unit1(x)
    x = x + x_t
    x_t = self.res_unit1(x)
    x = x + x_t
    x_t = self.res_unit2(x)
    x = self.identity(x) + x_t
    x_t = self.res_unit3(x)
    x = x + x_t
    x_t = self.res_unit3(x)
    x = x + x_t
    x_t = self.res_unit3(x)
    x = x + x_t
    x = self.flatten_layer(x)
    x = self.classifier(x)
    probs = tf.nn.softmax(x)
    return probs

In [79]:
model = cnn_model()

model optimizer, loss등 정의 compile

In [80]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train

In [81]:
model.fit(train_img, train_label, epochs=5)

Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 4.3760 - accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 24ms/step - loss: 12.2463 - accuracy: 0.5000
Epoch 3/5
1/1 [==============================] - 0s 26ms/step - loss: 2.5279 - accuracy: 0.7500
Epoch 4/5
1/1 [==============================] - 0s 21ms/step - loss: 2.9802e-08 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 22ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [72]:
model.summary()

Model: "cnn_model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_126 (Sequential)  (None, 64, 64, 12)       1776      
                                                                 
 sequential_127 (Sequential)  (None, 64, 64, 12)       2712      
                                                                 
 sequential_128 (Sequential)  (None, 64, 64, 12)       2712      
                                                                 
 sequential_129 (Sequential)  (None, 64, 64, 12)       2712      
                                                                 
 sequential_130 (Sequential)  (None, 32, 32, 24)       8016      
                                                                 
 sequential_131 (Sequential)  (None, 32, 32, 24)       10608     
                                                                 
 sequential_132 (Sequential)  (None, 32, 32, 24)      

#Dump

In [29]:
model(train_img)

(4, 64, 64, 12)
(4, 64, 64, 12)


<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[0.01513817, 0.87859064, 0.00146113, 0.10481004],
       [0.02088634, 0.9301859 , 0.00163644, 0.04729134],
       [0.05060832, 0.80482614, 0.00253833, 0.14202717],
       [0.01934402, 0.88991696, 0.00185118, 0.08888792]], dtype=float32)>

In [ ]:
# def residual_unit(self, x, in_channel, out_channel, kernel_size):
  #   if in_channel == out_channel:
  #     res1 = layers.Conv2D(out_channel, kernel_size, padding='same', strides=(1, 1))
  #   else:
  #     res1 = layers.Conv2D(out_channel, kernel_size, padding='same', strides=(2, 2))
  #   batch1 = layers.BatchNormalization()
  #   relu1 = layers.ReLU()
  #   res2 = layers.Conv2D(out_channel, kernel_size, padding='same')
  #   batch2 = layers.BatchNormalization()
  #   relu2 = layers.ReLU()

  #   x_t = res1(x)
  #   x_t = batch1(x_t)
  #   x_t = relu1(x_t)
  #   x_t = res2(x_t)
  #   x_t = batch2(x_t)
  #   x_t = relu2(x_t)

  #   if in_channel != out_channel:
  #     identity = layers.Conv2D(out_channel, (1, 1), padding='same', strides=(2, 2))
  #     x = identity(x)
  #   print(x_t.shape)
  #   print(x.shape)
  #   return x_t + x

  # channels = x.shape[3]
    # for l in range(self.num_residual_layers):
    #   for u in range(self.num_residual_units):
    #     x = self.residual_unit(x, channels, (lambda x,y:channels*2 if x != 0 and y == 0 else channels)(l, u), self.res_kernel_size)
    #     channels = channels * 2 if l != 0 and u == 0 else channels

In [ ]:
# model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape = (256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# model information
model.summary()

In [ ]:
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#print(model(train_img))

In [ ]:
# in_channel : input image channel size, out_channel : output image channel of each unit
# kernel_size  ex) (3, 3), (5, 5), ..
class ResidualUnit(tf.keras.Model):
  def __init__(self, in_channel, out_channel, kernel_size, stride=False):
    super(ResidualUnit, self).__init__()
    if stride:
      self.conv1 = layers.Conv2D(out_channel, kernel_size, padding='same', activation='relu', stride=(2, 2))
    else:
      self.conv1 = layers.Conv2D(out_channel, kernel_size, padding='same', activation='relu')
    self.conv2 = layers.Conv2D(out_channel, kernel_size, padding='same', activation='relu')

    if in_channel ==  out_channel:
      self.identity = lambda x: x
    else:
      self.identity = layers.Conv2D(out_channel, (1, 1), padding='same')

  def call(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    return x + self.identity(x)

In [ ]:
class ResidualLayer(tf.keras.Model):
  def __init__(self, in_channel, out_channel, kernel_size, stride=False):
    super(ResidualLayer, self).__init__()
    # residualunit + unit + unit
    self.res1 = ResidualUnit(in_channel, out_channel, kernel_size, stride)
    self.res2 = ResidualUnit(out_channel, out_channel, kernel_size)
    self.res3 = ResidualUnit(out_channel, out_channel, kernel_size)
  
  def call(self, x):
    x = self.res1(x)
    x = self.res2(x)
    x = self.res3(x)
    return x

In [68]:
class cnn_model(tf.keras.Model):
  global input_img_shape, input_channel, layer1_output_channel, layer1_kernel_size, res_kernel_size, layer2_output_channel, layer3_output_channel
  def __init__(self):
    super(cnn_model, self).__init__()
    self.num_residual_units = 3 # number of units per layer
    self.num_residual_layers = 2 # number of layer per model
    self.res_kernel_size = (3, 3)

    self.layer1 = models.Sequential(
        [layers.Conv2D(layer1_output_channel, layer1_kernel_size, strides=(2, 2), 
                       padding='same', input_shape=input_img_shape, activation='relu'),
         layers.MaxPool2D(pool_size=(2, 2))]
    )

    self.unit1 = models.Sequential(
        [layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.unit2 = models.Sequential(
        [layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.unit3 = models.Sequential(
        [layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.unit4 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, strides=(2, 2), padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.unit5 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.unit6 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.unit7 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )   

    self.flatten_layer = layers.Flatten()

    self.classifier = models.Sequential(
        [layers.Dense(4)]
    )

    self.identity = layers.Conv2D(layer3_output_channel, (1, 1), padding='same', strides=(2, 2))

  def call(self, x):
    x = self.layer1(x)
    x_t = self.unit1(x)
    x = x + x_t
    x_t = self.unit2(x)
    x = x + x_t
    x_t = self.unit3(x)
    x = x + x_t
    x_t = self.unit4(x)
    x = self.identity(x) + x_t
    x_t = self.unit5(x)
    x = x + x_t
    x_t = self.unit6(x)
    x = x + x_t
    x_t = self.unit7(x)
    x = x + x_t
    x = self.flatten_layer(x)
    x = self.classifier(x)
    probs = tf.nn.softmax(x)
    return probs